# Construindo o Darknet

In [ ]:
!sudo apt-get install build-essential git libopencv-dev cmake

!nvidia-smi
!nvcc --version

!git clone https://github.com/hank-ai/darknet

# Build Darknet/YOLO
%mkdir -p /content/darknet/build
%cd /content/darknet/build
!cmake -DCMAKE_BUILD_TYPE=Release ..
%cd /content/darknet/build
!make -j $(nproc) package


!sudo dpkg -i darknet*.deb

!darknet --version

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.15).
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
/bin/bash: line 1: nvidia-smi: command not found
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Cloning into 'darknet'...
remote: Enumerating objects: 20495, done.
remote: Counting objects: 100% (1305/1305), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 20495 (delta 1176), reused 1094 (delta 1049), pack-reused 19190 (from 3)
Receiving objects: 100% (20495/20495), 21.23 MiB | 20.60 MiB/s, done.


In [ ]:
!git clone https://github.com/jpierreribeiro/deteccaodepragas/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Treino

In [ ]:
!darknet detector -map -dont_show train /content/btabaci/btabaci.data /content/btabaci/btabaci.cfg | tee output.log

In [ ]:
#Mostrar os resultados...
import cv2 as cv2
from matplotlib import pyplot as plt
img = cv2.imread('chart.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Checando resultados

Buildando DarkHelp

In [ ]:
!sudo apt-get install build-essential libtclap-dev libmagic-dev libopencv-dev
%cd /content

!git clone https://github.com/stephanecharette/DarkHelp.git
%cd /content/DarkHelp
%mkdir build
%cd build
!cmake -DCMAKE_BUILD_TYPE=Release ..
!make -j $(nproc) package
!sudo dpkg -i darkhelp*.deb
%cd ~
!DarkHelp --version

In [ ]:
!DarkHelp /content/btv4/btv4.cfg /content/btv4/tabaci.names /content/drive/MyDrive/artigocomputervision/btv4_best.weights bm.mp4
#img = cv2.imread('predictions.jpg')
#plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Utilizando DarkHelp para analises com melhor desempenho.

In [ ]:
%cd /content/testing

import DarkHelp
import json
from matplotlib import pyplot as plt

print("DarkHelp v" + DarkHelp.DarkHelpVersion().decode())
print("Darknet v" + DarkHelp.DarknetVersion().decode())

dh = DarkHelp.CreateDarkHelpNN(
    "arquivo.cfg".encode(),
    "arquivo.weights".encode(),
    "arquivo.names".encode())
DarkHelp.EnableAnnotationAutoHideLabels(dh, False)
DarkHelp.EnableNamesIncludePercentage(dh, False)
DarkHelp.EnableSnapping(dh, False)
DarkHelp.EnableTiles(dh, False)
DarkHelp.SetThreshold(dh, 0.20)
DarkHelp.SetAnnotationLineThickness(dh, 1)

DarkHelp.PredictFN(dh, "foto.jpg".encode())

DarkHelp.Annotate(dh, "output.jpg".encode())
img = plt.imread("output.jpg")
plt.imshow(img)

j = json.loads(DarkHelp.GetPredictionResults(dh))
for prediction in j['file'][0]['prediction']:
    if False:
        print(prediction)
    idx = prediction['best_class']
    name = prediction['name']
    prob = prediction['best_probability'] * 100.0
    x = prediction['rect']['x']
    y = prediction['rect']['y']
    w = prediction['rect']['width']
    h = prediction['rect']['height']
    print(f"class #{idx}: {prob:.1f}% \"{name}\" at {prediction['rect']}")

    if False:
        roi = img[y:y+h, x:x+w]
        plt.imshow(roi)

DarkHelp.DestroyDarkHelpNN(dh)
